In [1]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_excel("./복합기 크롤링 중복제거.xlsx")

In [3]:
cols = ['Unnamed: ' + str(i) for i in range(3,30)]
data = data.drop(cols, axis=1)
cols2=['스펙 목록', '가격']
data = data.drop(cols2, axis=1)

In [4]:
data['productid'] = [i for i in range(0,5901)]

In [5]:
test={'user':['user' + str(np.random.randint(0,30)) for i in range(1,5902)],
     'rating':[np.random.randint(0,6) for i in range(1,5902)],
     'productid':[np.random.randint(0,5902) for i in range(1,5902)]}
test = pd.DataFrame(test)
test.head()

,user,rating,productid
0,user24,4,899
1,user8,2,5047
2,user13,5,4541
3,user9,4,3319
4,user0,0,2559


In [6]:
merge_data = pd.merge(test, data, on = 'productid')

In [7]:
merge_data

,user,rating,productid,상품명
0,user24,4,899,Canon MF643Cdw
1,user8,2,5047,삼성전자 SL-C563FW
2,user20,0,5047,삼성전자 SL-C563FW
3,user25,0,5047,삼성전자 SL-C563FW
4,user13,5,4541,삼성전자 MultiXpress SL-X3220NR
...,...,...,...,...
5893,user3,4,3331,HP 컬러 레이저젯 엔터프라이즈 M751dn
5894,user26,1,1723,Canon R762KG
5895,user27,1,4013,삼성전자 CLX-7455(AF)
5896,user1,1,2015,Epson Stylus Photo TX700W 무한공급기


In [8]:
rating = merge_data.pivot_table('rating', index='user', columns='상품명').fillna(0)
rating.head()

상품명,ApeosPort 2560,ApeosPort 3060,ApeosPort 3560,ApeosPort C2060,ApeosPort C2560,ApeosPort C3060,ApeosPort-II C3000,ApeosPort-IV C2270,ApeosPort-IV C2270CP,ApeosPort-IV C2270CPF,...,코니카미놀타 MagiColor 4690MF,코니카미놀타 bizhub 164,코니카미놀타 bizhub 211,코니카미놀타 bizhub 283,코니카미놀타 bizhub 362,코니카미놀타 bizhub 363,코니카미놀타 bizhub 423,코니카미놀타 bizhub C200,코니카미놀타 bizhub C220,코니카미놀타 bizhub C360
user,,,,,,,,,,,,,,,,,,,,,
user0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
 #matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = rating.values

# user_ratings_mean은 사용자의 평균 평점 
user_ratings_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-영화에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [10]:
matrix

array([[0., 0., 3., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [11]:
matrix.shape

(30, 1285)

In [12]:
user_ratings_mean.shape

(30,)

In [13]:
matrix_user_mean.shape

(30, 1285)

In [14]:
pd.DataFrame(matrix_user_mean, columns = rating.columns).head()

상품명,ApeosPort 2560,ApeosPort 3060,ApeosPort 3560,ApeosPort C2060,ApeosPort C2560,ApeosPort C3060,ApeosPort-II C3000,ApeosPort-IV C2270,ApeosPort-IV C2270CP,ApeosPort-IV C2270CPF,...,코니카미놀타 MagiColor 4690MF,코니카미놀타 bizhub 164,코니카미놀타 bizhub 211,코니카미놀타 bizhub 283,코니카미놀타 bizhub 362,코니카미놀타 bizhub 363,코니카미놀타 bizhub 423,코니카미놀타 bizhub C200,코니카미놀타 bizhub C220,코니카미놀타 bizhub C360
0,-0.237484,-0.237484,2.762516,1.762516,-0.237484,-0.237484,-0.237484,-0.237484,-0.237484,-0.237484,...,-0.237484,-0.237484,-0.237484,-0.237484,-0.237484,-0.237484,-0.237484,-0.237484,-0.237484,-0.237484
1,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633,...,2.727367,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633,-0.272633
2,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,...,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571,-0.247571
3,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,...,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384,-0.264384
4,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,...,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807,-0.232807


In [15]:
# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [16]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(30, 12)
(12,)
(12, 1285)


In [17]:
sigma = np.diag(sigma)

In [18]:
sigma[0]

array([28.83903407,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

In [19]:
sigma[1]

array([ 0.        , 29.36789551,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

In [171]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [172]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = rating.columns)
df_svd_preds.head()

상품명,ApeosPort 3560,ApeosPort C2560,ApeosPort C3060,ApeosPort-IV C2270,ApeosPort-IV C2270CPF,ApeosPort-IV C2270CPS,ApeosPort-IV C4430,ApeosPort-IV C4470,ApeosPort-IV C4470 CP,ApeosPort-IV C4470 CPF,...,코니카미놀타 C036DN,코니카미놀타 bizhub 164,코니카미놀타 bizhub 211,코니카미놀타 bizhub 215,코니카미놀타 bizhub 362,코니카미놀타 bizhub 363,코니카미놀타 bizhub 423,코니카미놀타 bizhub C200,코니카미놀타 bizhub C220,코니카미놀타 bizhub C360
0,-0.130609,-0.071996,-0.103522,-0.361168,0.134467,0.007515,0.007515,0.102929,-0.222692,0.025204,...,-0.234472,0.025088,0.242647,0.175160,0.167645,0.306855,0.175160,-0.591611,0.007515,0.207659
1,0.230667,-0.022495,0.429130,0.067887,0.254392,-0.010292,-0.010292,0.175614,0.391306,0.105614,...,0.210085,0.295517,-0.445446,0.020957,0.224639,-0.436925,0.020957,0.162971,-0.010292,-0.062295
2,2.586430,0.050507,-0.150891,0.098314,0.117500,0.013560,0.013560,0.291493,4.301676,0.133942,...,0.384295,0.181188,0.355479,0.023888,0.093880,0.508514,0.023888,0.294335,0.013560,-0.241860
3,-0.250712,0.174044,0.054247,0.121489,0.173652,0.021551,0.021551,-0.174498,-0.432221,0.184137,...,0.491375,0.602742,-0.302840,0.208946,0.033926,-0.385799,0.208946,0.216964,0.021551,-0.197602
4,-0.192544,-0.007409,0.263731,0.271286,-0.509586,-0.000979,-0.000979,1.202660,-0.320254,0.283432,...,0.436145,1.163487,0.165953,-0.009852,-0.013979,0.262490,-0.009852,0.094161,-0.000979,1.607636


In [173]:
df_svd_preds.shape

(30, 1302)

In [174]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id - 1 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user == 'user'+str(user_id)]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    user_history = user_data.merge(ori_movies_df, on = 'productid').sort_values(['rating'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_movies_df[~ori_movies_df['productid'].isin(user_history['productid'])]
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'productid')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [148]:
already_rated, predictions = recommend_movies(df_svd_preds, 29, data, test, 10)

KeyError: 'productid'